In [ ]:
!pip install catboost
from google.colab import drive
drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 13.7 MB/s eta 0:00:00
Mounted at /content/drive


In [ ]:
# Core libraries for data handling
import numpy as np
import pandas as pd

# Machine learning models and utilities
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier

# For handling imbalanced datasets
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE, RandomOverSampler

# Data preprocessing and pipeline utilities
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler


class DataPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self, scaler_type='standard'):
        self.num_transformer = None
        self.cat_transformer = None
        self.preprocessor = None
        self.scaler = {
            'standard': StandardScaler(),
            'minmax': MinMaxScaler(),
            'robust': RobustScaler()
        }.get(scaler_type, StandardScaler())

    def fit(self, X):
        num_cols = X.select_dtypes(include=['int64', 'float64']).columns
        cat_cols = X.select_dtypes(include=['object']).columns
        self.num_transformer = Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', self.scaler)
        ])
        self.cat_transformer = Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('onehot', OneHotEncoder(handle_unknown='ignore'))
        ])
        self.preprocessor = ColumnTransformer(
            transformers=[
                ('num', self.num_transformer, num_cols),
                ('cat', self.cat_transformer, cat_cols)
            ])
        self.preprocessor.fit(X)
        return self

    def transform(self, X):
        return self.preprocessor.transform(X)

    def fit_transform(self, X):
        self.fit(X)
        return self.transform(X)


class DataBalancer:
    def __init__(self, method='SMOTE', random_state=None):
        if method not in ['SMOTE', 'ROS', 'RUS']:
            raise ValueError("Method must be 'SMOTE', 'ROS', or 'RUS'")
        self.method = method
        self.random_state = random_state

    def balance_data(self, X, y):
        balancer = {
            'SMOTE': SMOTE(random_state=self.random_state),
            'ROS': RandomOverSampler(random_state=self.random_state),
            'RUS': RandomUnderSampler(random_state=self.random_state)
        }.get(self.method, SMOTE(random_state=self.random_state))
        X_balanced, y_balanced = balancer.fit_resample(X, y)
        return X_balanced, y_balanced


class FairnessEvaluator:
    def __init__(self, y_true, y_pred, sensitive_features):
        self.y_true = np.array(y_true)
        self.y_pred = np.array(y_pred)
        self.sensitive_features = np.array(sensitive_features).flatten()
        self.majority_group, self.minority_group = self.identify_groups()

    def identify_groups(self):
        unique_groups, counts = np.unique(self.sensitive_features, return_counts=True)
        majority_group = unique_groups[np.argmax(counts)]
        minority_group = unique_groups[np.argmin(counts)]
        return majority_group, minority_group

    def disparate_impact(self):
        majority_positive_rate = np.mean(self.y_pred[self.sensitive_features == self.majority_group])
        minority_positive_rate = np.mean(self.y_pred[self.sensitive_features == self.minority_group])
        return minority_positive_rate / majority_positive_rate

    def statistical_parity_difference(self):
        majority_positive_rate = np.mean(self.y_pred[self.sensitive_features == self.majority_group])
        minority_positive_rate = np.mean(self.y_pred[self.sensitive_features == self.minority_group])
        return minority_positive_rate - majority_positive_rate

    def equal_opportunity_difference(self):
        majority_true_positive_rate = np.mean(self.y_pred[(self.sensitive_features == self.majority_group) & (self.y_true == 1)])
        minority_true_positive_rate = np.mean(self.y_pred[(self.sensitive_features == self.minority_group) & (self.y_true == 1)])
        return minority_true_positive_rate - majority_true_positive_rate

    def average_odds_difference(self):
        majority_fpr = np.mean(self.y_pred[(self.sensitive_features == self.majority_group) & (self.y_true == 0)])
        minority_fpr = np.mean(self.y_pred[(self.sensitive_features == self.minority_group) & (self.y_true == 0)])
        fpr_diff = minority_fpr - majority_fpr
        majority_tpr = np.mean(self.y_pred[(self.sensitive_features == self.majority_group) & (self.y_true == 1)])
        minority_tpr = np.mean(self.y_pred[(self.sensitive_features == self.minority_group) & (self.y_true == 1)])
        tpr_diff = minority_tpr - majority_tpr
        return (fpr_diff + tpr_diff) / 2

    def evaluate_fairness(self, metric):
        metrics = {
            'DI': self.disparate_impact,
            'SPD': self.statistical_parity_difference,
            'EOD': self.equal_opportunity_difference,
            'AOD': self.average_odds_difference
        }
        return metrics[metric]()


class FairBoostNet:
    def __init__(self, train_age, fairness_metric):
        self.thereshhold = 0.5
        self.sens_train = train_age
        self.layer_one_models = [XGBClassifier(random_state=random_state, verbosity=0),
                                 GradientBoostingClassifier(random_state=random_state),
                                 AdaBoostClassifier(random_state=random_state),
                                 CatBoostClassifier(random_state=random_state, silent=True)]
        self.meta_model = XGBClassifier(random_state=random_state, verbosity=0)
        self.fairness_metric = fairness_metric
        self.attention_scores = None

    def fit(self, X, y):
        layer_one_fairness, layer_one_predictions = [], []
        for model in self.layer_one_models:
            model.fit(X, y)
            predictions = model.predict(X)
            layer_one_predictions.append(predictions)
            fairness_eval_train = FairnessEvaluator(y, predictions, self.sens_train)
            layer_one_fairness.append(fairness_eval_train.evaluate_fairness(self.fairness_metric))
        layer_one_predictions = np.array(layer_one_predictions).T
        fairness_scores = 1 - np.abs(np.array(layer_one_fairness))
        self.attention_scores = fairness_scores / np.sum(fairness_scores)
        weighted_predictions = layer_one_predictions * self.attention_scores
        self.meta_model.fit(weighted_predictions, y)
        return

    def predict(self, X):
        layer_one_predictions = []
        for model in self.layer_one_models:
            predictions = model.predict(X)
            layer_one_predictions.append(predictions)
        layer_one_predictions = np.array(layer_one_predictions).T
        if self.attention_scores is None:
            raise ValueError("Attention scores have not been set. Please train the model first.")
        weighted_predictions = layer_one_predictions * self.attention_scores
        final_predictions = self.meta_model.predict(weighted_predictions)
        return final_predictions

def evaluate_model(model, X_train, y_train, X_test, y_test, X_train_age, X_test_age, fairness_metric):
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    test_metrics = {
        'Confusion Matrix': confusion_matrix(y_test, predictions),
        'Accuracy': accuracy_score(y_test, predictions),
        'Precision': precision_score(y_test, predictions, zero_division=0),
        'Recall': recall_score(y_test, predictions),
        'F1': f1_score(y_test, predictions)
    }
    fairness_eval_test = FairnessEvaluator(y_test, predictions, X_test_age)
    test_metrics['Fairness DI'] = fairness_eval_test.disparate_impact()
    test_metrics['Fairness EOD'] = fairness_eval_test.evaluate_fairness(fairness_metric)
    test_metrics['Fairness SPD'] = fairness_eval_test.statistical_parity_difference()
    test_metrics['Fairness AOD'] = fairness_eval_test.average_odds_difference()
    return test_metrics

def fairboostnet(X_train, y_train, X_test, y_test, X_train_age, X_test_age, fairness_metric):
    fairboost_model = FairBoostNet(X_train_age, fairness_metric)
    fairboost_model.fit(X_train, y_train)
    predictions = fairboost_model.predict(X_test)
    fairness_eval_test = FairnessEvaluator(y_test, predictions, X_test_age)
    test_metrics = evaluate_model(fairboost_model, X_train, y_train, X_test, y_test, X_train_age, X_test_age, fairness_metric)
    test_metrics['Fairness DI'] = fairness_eval_test.disparate_impact()
    test_metrics['Fairness EOD'] = fairness_eval_test.evaluate_fairness(fairness_metric)
    test_metrics['Fairness SPD'] = fairness_eval_test.statistical_parity_difference()
    test_metrics['Fairness AOD'] = fairness_eval_test.average_odds_difference()
    return test_metrics


# Setting Random State
random_state = 2024
np.random.seed(random_state)
# Load dataset
dataset = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CSI 5137/Project/bank-additional-full.csv', delimiter=';')
dataset['age'] = (dataset['age'] > 35).astype(int)
# Separating features and target variable
data = dataset.drop('y', axis=1)
labels = dataset['y'].map({'yes': 1, 'no': 0})
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, stratify=labels, random_state=random_state)
# Extract gender information for fairness evaluation
age_train, age_test = X_train['age'], X_test['age']
# Drop 'CODE_GENDER' column if not needed for further processing
X_train = X_train.drop(['age'], axis=1)
X_test = X_test.drop(['age'], axis=1)
# Preprocess the dataset
preprocessor = DataPreprocessor(scaler_type='minmax')
preprocessed_X_train = preprocessor.fit_transform(X_train)
preprocessed_X_test = preprocessor.transform(X_test)
updated_preprocessed_X_train = np.concatenate([age_train.to_numpy().reshape(-1, 1), preprocessed_X_train], axis=1)
updated_preprocessed_X_test = np.concatenate([age_test.to_numpy().reshape(-1, 1), preprocessed_X_test], axis=1)
# Instantiate Data Balancer and balance the training data
data_balancer = DataBalancer(method='SMOTE', random_state=random_state)
balanced_X_train, balanced_y_train = data_balancer.balance_data(updated_preprocessed_X_train, y_train)
# Extract sensitive attribute
X_train_age = balanced_X_train[:, [0]]
X_test_age = updated_preprocessed_X_test[:, [0]]
fairness_metric = 'EOD'
models = {
    'FairBoostNet': fairboostnet,
    'XGBoost': XGBClassifier(random_state=random_state, verbosity=0),
    'Gradient Boosting': GradientBoostingClassifier(random_state=random_state),
    'AdaBoost': AdaBoostClassifier(random_state=random_state),
    'CatBoost': CatBoostClassifier(random_state=random_state, silent=True)
}
print("================================================================")
for name, model_function in models.items():
    if name == 'FairBoostNet':
        test_metrics = model_function(balanced_X_train, balanced_y_train, updated_preprocessed_X_test, y_test, X_train_age, X_test_age, fairness_metric)
    else:
        model_function.fit(balanced_X_train, balanced_y_train)
        test_metrics = evaluate_model(model_function, balanced_X_train, balanced_y_train, updated_preprocessed_X_test, y_test, X_train_age, X_test_age, fairness_metric)
    print(f"Model: {name}")
    for metric_name, metric_value in test_metrics.items():
        if metric_name == 'Confusion Matrix':
            print(f"{metric_name}:")
            print(f"TP: {metric_value[1, 1]} \t FP: {metric_value[0, 1]}")
            print(f"FN: {metric_value[1, 0]} \t TN: {metric_value[0, 0]}")
        else:
            print(f"{metric_name}: {metric_value}")
    print("================================================================")

Model: FairBoostNet
Confusion Matrix:
TP: 526 	 FP: 298
FN: 402 	 TN: 7012
Accuracy: 0.9150279193979121
Precision: 0.6383495145631068
Recall: 0.5668103448275862
F1: 0.6004566210045663
Fairness DI: 1.3539555259086526
Fairness EOD: 0.015257583075054648
Fairness SPD: 0.031012291120689317
Fairness AOD: 0.01700462879186705
Model: XGBoost
Confusion Matrix:
TP: 561 	 FP: 345
FN: 367 	 TN: 6965
Accuracy: 0.9135712551590192
Precision: 0.6192052980132451
Recall: 0.6045258620689655
F1: 0.6117775354416577
Fairness DI: 1.3725946242738973
Fairness EOD: 0.024818189064617302
Fairness SPD: 0.03566112045357213
Fairness AOD: 0.02336176324339507
Model: Gradient Boosting
Confusion Matrix:
TP: 761 	 FP: 726
FN: 167 	 TN: 6584
Accuracy: 0.8915999028890508
Precision: 0.511768661735037
Recall: 0.8200431034482759
F1: 0.6302277432712217
Fairness DI: 1.2668570370281145
Fairness EOD: 0.04526166239198015
Fairness SPD: 0.04352221202846221
Fairness AOD: 0.03445747876551497
Model: AdaBoost
Confusion Matrix:
TP: 707 	 